In [1]:
import numpy as np
import pandas as pd

In [2]:
A = .0001234 
B = .000004
c = 1.122

In [3]:
pd.set_option('display.float_format', lambda x: '%.4f' % x)

In [134]:
class makehams:
    def __init__(self, a, b, c, l_0 = 1_000_000, w = 120, 
                 select = {0:.5, 1:.75}):
        self.a = a
        self.b = b
        self.c = c
        self.l_0 = l_0
        self.w = w
        self.select = select
        self.mu_list = pd.Series({age:self.mu(age) for age in range(0, w+1)})
        self.S_list = pd.Series({age:self.S(age) for age in range(0, w+1)})
        self.q_list = pd.Series({age:self.q(age,0,1) for age in range(0, w+1)})
        self.p_list = pd.Series({age:self.p(0, age) for age in range(0, w+1)})
        self.l_list = pd.Series({age:self.l(age) for age in range(0, w+1)})
        self.e_list = pd.Series({age:self.l(age) for age in range(0, w+1)})
        
    def mu (self, x): 
        mu_x = self.a + self.b * self.c ** x
        return mu_x
        
    def S (self, x):
        a = self.a; b = self.b; c = self.c
        s_x = np.e ** ( - (a * x) - ( b * (c ** x) - b ) / np.log(c) )
        return s_x
                   
    def q (self, x, t, u):
        tbaru_q_x = ( self.S(x + t) - self.S(x + t + u) ) / self.S(x)
        if x + t + u >= 121:
            tbaru_q_x = 1
        return tbaru_q_x
        
    def p (self, x,t):
        a = self.a; b = self.b; c = self.c
        t_p_x = np.e ** ( -a * t - b/np.log(c) * c ** x * (c ** t - 1) ) 
        return t_p_x
        
    def l (self, x):
        l_x = self.l_0 * self.p(0, x)
        return l_x
        
    def t_p_x_list (self, x):
        """return list of consecutive t_p_x values"""
        w = self.w
        next_age = int(x + 1)
        tps = {}
        for t in range (0, w - next_age + 1):
            tps[f'{np.round(t + (next_age - x),2)}_p_{x}'] = \
            self.p(x, t + (next_age - x))
            
        tps[f'{np.round(w - x + 1, 2)}_p_{x}'] = 0
        return tps

    def p_x_list (self, x):
        w = self.w
        next_age = int(x + 1)
        ps = {}
        ps[f'p_{x}'] = self.p(x, next_age - x)
        for t in range (0, w - next_age):
            ps[f'p_{next_age + t}'] = self.p(next_age + t, 1)
        ps[f'p_{w}'] = 0
        return ps
        
    def mu_x_list(self,x):
        return {f'mu_{x + t}': self.mu(x+t) for t in range(0,self.w - x + 1)}
        
    def q_x_list(self, x):
        return {f'q_{x + t}': self.q(x + t, 0, 1) for 
                t in range(0, self.w - x + 1)}
        
    def l_x_list(self, x):
        return dict(zip(
            [f'l_{x+t}' for t in range(0, self.w - x + 1)],
            self.l_0*np.array([1] + list(self.t_p_x_list(x).values())[:-1])
        ))
    def e_x(self, x):
        return sum(list(self.t_p_x_list(x).values()))
        
    def e_x_list(self, x):
        w = self.w
        next_age = int(x + 1)
        curt_life_list = {}
        curt_life_list[f'e_{x}'] = self.e_x(x)
        
        for t in range(0, w - next_age + 1):                                    
            curt_life_list[f'e_{next_age+t}'] = \
                                        sum(list(
                                        self.t_p_x_list(next_age+t).values()))
        return curt_life_list

    
        
    def stats_table(self, x):
        table = pd.DataFrame({
            'mu_x': self.mu_x_list(x).values(),
            'p_x': self.p_x_list(x).values(),
            'q_x': self.q_x_list(x).values(),
            'l_x': self.l_x_list(x).values(),
            'e_x': self.e_x_list(x).values()
        }, index = list(range(x, self.w + 1))).round(4)
        table.index.name = 'age'
        return table
        
    def q_select(self, x):
        return {f'q[x]{s}': self.q(x,0,1)*mod if x < self.w else 1 for 
                s, mod in self.select.items()}
    def p_select(self, x):
        q_sel = self.q_select(x)
        return {f'p[x]{s}': 1 - q_sel[f'q[x]{s}']
               for s in self.select.keys()}
    # def e_x_select(age, select_yr):
    #     table = self.l_select_ult_table(age-select_yr)
    #     initial_lives = table.loc[age - select_yr,select_yr]
    #     table.loc[age,]             
    def p_select_table(self, min_age):
        sel_len = len(self.select)
        table = pd.concat([
            # there will be a 'square' of nan values 
            pd.DataFrame.from_dict(data = { leadup_yr: 
                        [np.nan]*sel_len for leadup_yr in 
         range(min_age-sel_len,min_age)}, orient = 'index'), 
            
            # pselect values for each year
            pd.DataFrame.from_dict({x: self.p_select(x).values() for x in 
         range(min_age, self.w + 1)}, orient = 'index')])
        table.columns = ['p[x]' if sel_yr==0 else f'p[x]+{sel_yr}' for sel_yr in self.select]
        return table

    def l_select_ult_table(self, min_age):
        select_len = len(self.select.keys())
        sel_index = list(range(min_age-select_len, self.w+1))
        sel_col_names = ['l[x]' if sel_yr == 0 else f'l[x]+{sel_yr}' 
                         for sel_yr in self.select.keys()]
        ult_col_name = f'lx+{select_len}'
        
        ult_col = pd.Series(
                    data = list(self.l_x_list(min_age - select_len).values()),
                    index = sel_index, name = f'lx+{select_len}')
        
        sel_columns = self.p_select_table(min_age).cumprod(axis='columns').loc[:,::-1].rdiv(
            ult_col.values, axis = 0)
        table = pd.concat([sel_columns, ult_col], axis = 1)
        table.columns = sel_col_names + [ult_col_name]
        
       
        # pd.Series(self.l_x_list(min_age - len(self.select.keys()))).values
        # table[sel_columns] = self.p_select_table(min_age).cumprod(
        #     axis = 'columns').mul(pd.Series(self.l_x_list(min_age)).values, 
        #                         axis = 'index').values
        # table.index = list(range(min_age,self.w+1))
        return table
        
    def q_select_table(self,min_age):
        sel_len = len(self.select)
        table = pd.concat([
            # there will be a 'square' of nan values 
            pd.DataFrame.from_dict(data = { leadup_yr: 
                        [np.nan]*sel_len for leadup_yr in 
         range(min_age-sel_len,min_age)}, orient = 'index'), 
            
            # pselect values for each year
            pd.DataFrame.from_dict({x: self.q_select(x).values() for x in 
         range(min_age, self.w + 1)}, orient = 'index')])
        table.columns = ['q[x]' if sel_yr==0 else f'q[x]+{sel_yr}' for sel_yr in self.select]
        return table
        # return pd.DataFrame.from_dict(
        #     {x: self.q_select(x) for x in 
        #      range(0, self.w + 1)}, orient = 'index')
    def e_x_select(self, age, sel_yr):
        select_ps = self.p_select_table(age-sel_yr).loc[
         age-sel_yr, self.p_select_table(age).columns[range(sel_yr, len(self.select))]]
        ultimate_ps = list(self.p_x_list(age+len(self.select)-sel_yr).values())
        
        return sum(np.cumprod(np.concatenate((select_ps, ultimate_ps))))
        
    def e_x_select_table(self, min_age):
        sel_len = len(self.select)
        table = pd.concat([
            # there will be a 'square' of nan values 
            pd.DataFrame.from_dict(data = {leadup_yr: 
                        [np.nan]*sel_len for leadup_yr in 
         range(min_age-sel_len,min_age)}, orient = 'index'), 
            
            # pselect values for each year
            pd.DataFrame.from_dict({x: [self.e_x_select(x - sel_yr,sel_yr) 
                for sel_yr in self.select] for x in range (min_age,self.w+1)},
            orient = 'index')])
        table.columns = ['e[x]' if sel_yr==0 else f'e[x]+{sel_yr}' for sel_yr in 
                         self.select]
        return table
        
    def select_table(self, age):
        return pd.concat([self.p_select_table(age), 
            self.q_select_table(age),self.l_select_ult_table(age),
                         self.e_x_select_table(age)], axis=1).loc[age:,:]
    def full_table(self,age):
        return pd.concat((self.stats_table(age), self.select_table(age)),axis=1)
mortality_model = makehams(A, B, c, 1_000_000)

In [125]:
mortality_model.e_x_select(21,1)

62.45726289969898

62.46267348465753

In [136]:
mortality_model.full_table(20).to_csv('./hw_3_solution_will_mohr.csv')

In [88]:

(mortality_model.e_x_select(20,0)[0].values,
mortality_model.e_x_select(20,0)[1])

(array([0.99991712, 0.99987568]),
 array([0.99982327, 0.99981676, 0.99980946, 0.99980126, 0.99979207,
        0.99978176, 0.99977019, 0.99975721, 0.99974264, 0.9997263 ,
        0.99970796, 0.99968739, 0.99966431, 0.99963841, 0.99960935,
        0.99957675, 0.99954018, 0.99949914, 0.9994531 , 0.99940144,
        0.99934349, 0.99927846, 0.99920551, 0.99912367, 0.99903185,
        0.99892884, 0.99881328, 0.99868363, 0.99853818, 0.99837502,
        0.99819198, 0.99798664, 0.99775631, 0.99749795, 0.99720814,
        0.99688307, 0.99651847, 0.99610955, 0.99565095, 0.99513664,
        0.9945599 , 0.9939132 , 0.99318811, 0.99237518, 0.99146386,
        0.99044237, 0.9892975 , 0.98801453, 0.98657702, 0.98496663,
        0.98316289, 0.98114304, 0.9788817 , 0.97635068, 0.97351866,
        0.97035092, 0.96680898, 0.96285031, 0.95842798, 0.9534903 ,
        0.9479805 , 0.9418364 , 0.93499011, 0.9273678 , 0.91888951,
        0.90946911, 0.89901435, 0.88742711, 0.87460391, 0.86043674,
        0.8448

In [ ]:
mortality_model.p_select_table(20)

In [ ]:
s = pd.Series(data = mortality_model.l_x_list(20).values(),index = range(20,mortality_model.w+1))
# s.index = range(20,mortality_model.w+1)
s

In [ ]:
mortality_model.p_select_table(20).cumprod(axis='columns')

In [ ]:
mortality_model.q_selects(20)

In [ ]:
mortality_model

In [ ]:
[(i,j) for i, j in enumerate([i**2 for i in range(0,5)])]

In [ ]:
mortality_model.stats_table(20).to_csv('./hw_2_python_soln.csv')